In [1]:
using LinearAlgebra, CSV, Random, Tables

In [48]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
k = 7;
# All close.
maximum(abs.(raw_data * raw_data .- data))

4.884981308350689e-15

In [2]:
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;

In [36]:
# node_id = 50
node_id = 300
sum(nodes[:, node_id] .== 1)

3

In [37]:
bounds[:, node_id]

3-element Array{Float64,1}:
 3.608137513865839
 6.26741838326191
 7.0

In [38]:
selected_data = findall(nodes[:, node_id] .== 1)

3-element Array{Int64,1}:
 58
 60
 62

In [39]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.6649665589270974

In [40]:
rank_one_term = (raw_data * u) .^ 2

101-element Array{Float64,1}:
 0.042608386423828526
 0.12169366148453958
 0.0010827657107646796
 0.21199488509712636
 0.3576958952379641
 0.590931332887428
 0.0028134344289715506
 0.045710787852978954
 0.0134758787384476
 0.0402507143194045
 ⋮
 0.19651099861244686
 0.03521501895354107
 0.00462342769518394
 0.026081129247667985
 0.0008624637095038933
 0.37524469637807883
 0.11466762911354514
 0.03909959370980699
 0.07896128909275592

In [41]:
lambda_1 = bounds[2, node_id]
lambda_2 = bounds[3, node_id] - bounds[1, node_id]
lambda_1 - lambda_2

2.8755558971277493

In [55]:
rank_n1_term = (lambda_1 - lambda_2) * (
    # 2 *
    mapslices(norm, sqrt(raw_data * (I - u * u') * raw_data); dims=2) .^ 2
    / (lambda_1 - lambda_2)
)

101×1 Array{Float64,2}:
 0.9573916135761725
 0.8783063385154616
 0.9989172342892357
 0.7880051149028751
 0.6423041047620378
 0.4090686671125715
 0.9971865655710256
 0.9542892121470192
 0.9865241212615488
 0.9597492856805928
 ⋮
 0.8034890013875533
 0.964784981046457
 0.9953765723048089
 0.973918870752333
 0.9991375362904977
 0.6247553036219169
 0.8853323708864563
 0.9609004062901878
 0.921038710907241

In [57]:
rank_one_term + rank_n1_term

101×1 Array{Float64,2}:
 1.000000000000001
 1.000000000000001
 1.0000000000000004
 1.0000000000000013
 1.0000000000000018
 0.9999999999999996
 0.9999999999999971
 0.9999999999999982
 0.9999999999999963
 0.9999999999999973
 ⋮
 1.0000000000000002
 0.9999999999999981
 0.9999999999999929
 1.0000000000000009
 1.0000000000000016
 0.9999999999999958
 1.0000000000000016
 0.9999999999999948
 0.9999999999999969